In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, lit, udf
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer
import string
import re
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import HashingTF, Tokenizer, IDF, PCA
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import pickle

def getSpark():
    return SparkSession.builder.master("local[8]").getOrCreate()

index = 1

spark = getSpark()
####Raw
df = spark.read.load(f'publisherRaw{index}/')
df = df.withColumn('label', df._hyperpartisan.cast('integer'))
testSet = spark.read.load(f'articleRaw{index}/')
testSet = testSet.withColumn('label', testSet._hyperpartisan.cast('integer'))

In [ ]:
ev = MulticlassClassificationEvaluator(metricName='accuracy')
crossval = CrossValidator(estimator=Pipeline(stages=[LogisticRegression(regParam=0.1, maxIter=20)]),
                          estimatorParamMaps=ParamGridBuilder().build(),
                          evaluator=ev,
                          numFolds=10, seed=42)
model = crossval.fit(testSet)
ev.evaluate(model.transform(df))

In [36]:
testSet.dropna().rdd.filter(lambda x: x.features is None).collect()


[]